In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
cd /content/gdrive/My Drive/EVA4/

/content/gdrive/My Drive/EVA4


In [0]:
!ls 

'fg_bg Images'


In [0]:
from zipfile import ZipFile
import zipfile
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import io
from itertools import groupby
import cv2
from tqdm.auto import tqdm
from pathlib import Path
from time import time
from torchvision import datasets
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.utils.data import random_split
from PIL import Image
import torchvision.transforms as T
import os
import torch
sns.set()

In [0]:
import zipfile

bgfgzip = zipfile.ZipFile('dataset_1_400','r')



In [0]:
datafiles = [info.filename for info in bgfgzip.infolist() if not info.is_dir()]

In [0]:
len(datafiles)

800000

In [0]:
for idx, file in enumerate(datafiles):
    imgdata = bgfgzip.read(file)
    img = Image.open(io.BytesIO(imgdata))
    img = img.convert("RGB")

    if idx % 1000 == 0:
        print(f'processed {file} to {img.size}')

processed OverlayDir/BgFg/bgfg312460.jpg to (224, 224)
processed OverlayDir/BgFg/bgfg285361.jpg to (224, 224)
processed OverlayDir/BgFg/bgfg242801.jpg to (224, 224)
processed OverlayDir/BgFg/bgfg257977.jpg to (224, 224)
processed OverlayDir/BgFg/bgfg51382.jpg to (224, 224)
processed OverlayDir/BgFg/bgfg344445.jpg to (224, 224)
processed OverlayDir/BgFg/bgfg257068.jpg to (224, 224)
processed OverlayDir/BgFg/bgfg199086.jpg to (224, 224)
processed OverlayDir/BgFg/bgfg345947.jpg to (224, 224)
processed OverlayDir/BgFg/bgfg241224.jpg to (224, 224)
processed OverlayDir/BgFg/bgfg122782.jpg to (224, 224)
processed OverlayDir/BgFg/bgfg32102.jpg to (224, 224)
processed OverlayDir/BgFg/bgfg223871.jpg to (224, 224)
processed OverlayDir/BgFg/bgfg328807.jpg to (224, 224)
processed OverlayDir/BgFg/bgfg390545.jpg to (224, 224)
processed OverlayDir/BgFg/bgfg115099.jpg to (224, 224)
processed OverlayDir/BgFg/bgfg270602.jpg to (224, 224)
processed OverlayDir/BgFg/bgfg193570.jpg to (224, 224)
processed Ov

In [0]:
class DenseDepth(Dataset):
    '''
    DenseDepth Dataset

    Input is fg_bg image AND bg image
    Target is fg_bg_mask AND depth_fg_bg image
    '''
    def __init__(self, root, data='BgFg', train=True, transform=None, target_transform=None, download=False):
        self.root = root
        self.transform = transform
        self.target_transform = target_transform

        # open the respective data and target
        
        bgfgzip =  ZipFile(os.path.join(self.root,'dataset_1_400'), 'r') # fg_bg
        
        bgfgpaths = [info.filename for info in bgfgzip.infolist() if not info.is_dir()]
       

        assert(len(bgfgpaths) == 800000 

        print(f'found {len(bgfgpaths)} fg_bg images')


        if data == 'BgFg':
            self.image_zip = bgfgzip
            self.image_paths = bgfgpaths
        elif data == 'bg':
            self.image_zip = bg_zip
            self.image_paths = bg_paths
        elif data == 'fg_bg_mask':
            self.image_zip = fg_bg_mask_zip
            self.image_paths = fg_bg_mask_paths
        elif data == 'depth_fg_bg':
            self.image_zip = depth_fg_bg_zip
            self.image_paths = depth_fg_bg_paths
        else:
            raise f'{data} is not a valid option'
        

        self.targets = []

        

    def __getitem__(self, index):
        imgdata = self.image_zip.read(self.image_paths[index])
        img = Image.open(io.BytesIO(imgdata))
        img = img.convert("RGB")
        img = np.array(img)

        if self.transform is not None:
            img = self.transform(img)
        
        return img


    def __len__(self):
        return len(self.image_paths)

In [0]:
dataset = DenseDepth(root='./', transform=T.Compose([T.ToTensor()]))

found 800000 fg_bg images


In [0]:
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=1)

In [0]:
def get_mean_and_std(dataset):
    '''Compute the mean and std value of dataset.'''
    dataloader = torch.utils.data.DataLoader(
        dataset, batch_size=1, shuffle=True, num_workers=1)
    mean = torch.zeros(3)
    std = torch.zeros(3)
    print('==> Computing mean and std..')
    for images in tqdm(dataloader):
        for i in range(3):
            mean[i] += images[:, i, :, :].mean()
            std[i] += images[:, i, :, :].std()
    mean.div_(len(dataset))
    std.div_(len(dataset))
    return mean, std

In [0]:
mean,std = get_mean_and_std(dataset)

==> Computing mean and std..


In [0]:
[f'{m:.15f}' for m in mean], [f'{s:.15f}' for s in std]

(['0.306218862533569', '0.287717372179031', '0.269325882196426'],
 ['0.214562639594078', '0.213071450591087', '0.213857620954514'])

***Submiited by G Uday Kiran, Srilakshmi V***